In [1]:
import pandas as pd
import numpy as np

In [33]:
def get_breadth_coverage(df):
    """ function gets dataframe created from output of command 
        "samtools bedtools coverage -a <BED> -b <BAM>"
        
    """
    
    for gene in df.gene.unique():
        percent= df[df.gene == gene].percent_covered.mean()
        cover["gene"].append(gene)
        cover["percent_covered"].append(round(percent * 100, 2))

    df = pd.DataFrame(cover).sort_values(by="gene", ascending=False) \
    .reset_index() \
    .drop("index", axis=1)
    
    return df

filename = "bedtools_coverage"
names = ["chr", "start", "end", "gene", "to_drop", "starnd", "n_reads", 
         "n_bases_covered", "length_of_gene_fragment", "percent_covered"]
cover = {"gene": [],
         "percent_covered": []}



df2 = pd.read_csv(filename, sep="\t", names=names).drop("to_drop", axis=1)

breadth = get_breadth_coverage(df2)

df2.head()




,chr,start,end,gene,starnd,n_reads,n_bases_covered,length_of_gene_fragment,percent_covered
0,chr1,12918805,12918926,PRAMEF2,+,0,0,121,0.0
1,chr1,12918861,12918973,PRAMEF2,+,0,0,112,0.0
2,chr1,12918942,12919066,PRAMEF2,+,0,0,124,0.0
3,chr1,12918987,12919110,PRAMEF2,+,0,0,123,0.0
4,chr1,12919089,12919215,PRAMEF2,-,0,0,126,0.0


In [6]:
df.head(2)

,chr,start,end,gene,starnd,n_reads,n_bases_covered,length_of_gene_fragment,percent_covered
0,chr1,12918805,12918926,PRAMEF2,+,0,0,121,0.0
1,chr1,12918861,12918973,PRAMEF2,+,0,0,112,0.0


In [44]:
import tempfile
import subprocess
target_regions_ = "DHS-003Z.primers-150bp.bed"
sample = "sorted_bashirli.bam"


def depth_calculation_in_target_regions(target_regions: str = target_regions_,
                                        patient: str = sample) -> pd.DataFrame:
    command = f"samtools bedcov {target_regions} {patient}".strip().split(' ')
    temp_file = tempfile.NamedTemporaryFile()

    samtools_bedcov = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding="utf-8")

    with open(temp_file.name, 'w')as temp:
        temp.write(samtools_bedcov.stdout)

    if not samtools_bedcov.stderr:
        names = ["chr", "start", "end", "gene", "to_drop", "strand", "depth"]
        df = pd.read_csv(temp_file, sep="\t", names=names).drop("to_drop", axis=1)

        return df

    raise PipelineError(
        f"an error occurred during the\n >>>> {' '.join(command)} <<<<\nERROR from {samtools_bedcov.stderr}")

    
def depth_calculation_for_genes(df):
    dct = {"gene": [],
           "mean_cov_depth_X": []}

    for gene in np.unique(df.gene.values):
        dct["gene"].append(gene)
        dct["mean_cov_depth_X"].append(df[df.gene == gene].depth.sum() /
                                       (df[df.gene == gene].end - df[df.gene == gene].start).sum())

    result = pd.DataFrame(dct)

    result = result.sort_values("gene", ascending=False) \
        .reset_index() \
        .drop('index', axis=1)

    result["mean_cov_depth_X"] = result["mean_cov_depth_X"].round(3)

    return result


depth = depth_calculation_in_target_regions()
depth = depth_calculation_for_genes(df=depth)

In [61]:
df.reset_index

AttributeError: module 'pandas' has no attribute 'reindex'

In [69]:
pd.concat([depth, breadth.percent_covered], axis=1) \
.sort_values("percent_covered", ascending=False).reset_index(drop=True)

,gene,mean_cov_depth_X,percent_covered
0,JAK2,2004.935,100.00
1,HRAS,659.639,100.00
2,PAX5,1597.056,100.00
3,CDKN2A,1504.344,100.00
4,ELANE,848.856,100.00
5,TERT,862.094,91.61
6,IL7R,513.714,89.45
7,TPMT,83.327,35.19
8,CREBBP,188.142,25.83
9,GJB3,0.641,19.70


In [29]:
df2

,gene,percent_covered
0,ZRSR2,14.58
1,XPO1,0.00
2,WT1,0.00
3,WRN,0.00
4,WAS,0.00
5,U2AF2,0.00
6,U2AF1,0.00
7,TUBA3C,0.00
8,TPMT,35.19
9,TP53,1.06


In [23]:
depth

,gene,mean_cov_depth_X
0,ZRSR2,14
1,XPO1,0
2,WT1,0
3,WRN,0
4,WAS,0
5,U2AF2,0
6,U2AF1,0
7,TUBA3C,0
8,TPMT,83
9,TP53,0
